# Chapter 1、2 介绍、MNIST(cifar10)全流程
> 目录  
> 1,numpy实现波士顿房价预测  
> 2,paddlle.fluid实现波士顿房价预测  
> 3,cifar10(简单CNN)  
>>   3.1 异步读取数据  
    3.2 分布式训练  
    3.3 动态学习率  
    3.4 VisualDL
    3.5 恢复训练
    
    

- 参数W与输入X组成的公式的基本结构称为假设H
- 模型假设H、评价函数(损失)、优化算法 是构成一个模型的三个部分
- 假设——>评价——>优化 
- 训练过程中的学习就是参数估计

&nbsp;

- (1) 神经网络思想的提出已经是75年前的事情了，但是直到2010年， 大数据涌现、硬件发展、算法优化 使得深度学习真正进入兴起时期。

基于深度学习的顶级会议 ICLR 统计，深度学习相关论文数量呈逐年递增的状态。


|ICML、KDD|与数据和模型技术相关的论文|
|--|--|
|CVPR|专注视觉|
|EMNLP|自然语言处理|

- (2) 实现了端到端的学习

在深度学习兴起之前，很多领域建模思路是投入大量精力做特征工程，将专家对某个领域的“人工理解”做成特征表达，然后使用简单的分类或回归网络完成任务。

但是在数据充足的情况下，深度学习模型可以实现端到端的学习，不需要做特征工程，将“原始特征”输入到“端到端模型”中，模型输出即可完成任务。

## 1,波士顿房价预测

[np.fromfile详解](https://zhuanlan.zhihu.com/p/136744114)

|||
|--|--|
|np.savetxt('a.csv',a,fmt='%.1f',delimiter=',')|将数组中的数据写入txt或者csv,多是逗号分隔|
|np.loadtxt('a.csv',dtype=np.int,delimiter=",")|读取csv或者txt文件中的数据到数组中|
|np.tofile("a.bat",sep=",",format='%d')|将数据压缩成一维再存储|
|np.fromfile("a.bat",dtype=np.int,sep=',').reshape(4,5)|读入二进制文件|


In [2]:
# (1)导入数据集
import json
import numpy as np
import matplotlib.pyplot as plt
'''
data = np.fromfile('data/data95203/housing.data',dtype=np.float,sep=' ')
#data是一维的数组，0-13是一个样本
data = data.reshape(data.shape[0]//14,14)
#这里的data是(506,14)，即506个样本

# (2)数据集划分
ratio = 0.8
offset = int(0.8*data.shape[0])
train_data = data[0:offset]

# (3)使用train_data的数据对所有数据进行归一化处理
maximums,minimums,avgs = train_data.max(axis=0),train_data.min(axis=0),train_data.sum(axis=0)/train_data.shape[0]
for i in range(14):
    data[:,i] = (data[:,i]-minimums[i])/(maximums[i]-minimums[i])
'''
#========================================================================================================================
# (4)封装成load_data函数
def load_data():
    feature_num = 14 #13个自变量拟合一个因变量
    data = np.fromfile('data/data95203/housing.data',dtype=np.float,sep=' ')
    data = data.reshape(data.shape[0]//feature_num,feature_num)
    ratio = 0.8
    offset = int(0.8*data.shape[0])
    train_data = data[0:offset]
    maximums,minimums,avgs = train_data.max(axis=0),train_data.min(axis=0),train_data.sum(axis=0)/train_data.shape[0]
    for i in range(14):
        data[:,i] = (data[:,i]-minimums[i])/(maximums[i]-minimums[i])
    train_data = data[:offset]
    test_data = data[offset:]
    return train_data,test_data
#=======================================================================================================================

# (5)模型设计
class LinearNetwork(object):
    def __init__(self,num_of_weights):
        #随机初始化权重
        np.random.seed(2022)
        self.w = np.random.randn(num_of_weights,1)
        self.b = 0
    def forward(self,x):
        z = np.dot(x,self.w) + self.b
        return z
    def loss(self,z,y):#残差平方损失函数
        #使用平方误差而不是绝对值有两个好处
            #1,曲线的最低点是可导的
            #2,越接近最低点，坡度逐渐放缓，有助于通过当前的梯度值判断接近最低点的程度
        error = z.squeeze() - y.squeeze()
        cost = np.sum(error*error)/404

        return cost
    def gradient(self,x,y):
        z = self.forward(x)
        gradient_w_tmp = np.dot((z.squeeze()-y),x)
        #print(gradient_w_tmp.shape)
        gradient_w = gradient_w_tmp/404
        gradient_b = np.mean(z-y)
        return gradient_w,gradient_b
    def update(self,gradient_w,gradient_b,lr):
        self.w = self.w.squeeze() - lr*gradient_w.squeeze()
        self.b = self.b - lr*gradient_b

    def train(self,x,y,iterations=100,lr=0.01):
        losses = []
        for i in range(iterations):
            z = self.forward(x)
            L = self.loss(z,y)
            #print(z.shape,L.shape)

            gradient_w,gradient_b = self.gradient(x,y)
            
            self.update(gradient_w,gradient_b,lr=0.01)
            losses.append(L)
            if (i+1)%10 == 0:
                print('iter:{} loss{}'.format(i,L))
        return losses

# (7)开始训练,训练iterations轮，即遍历数据集iterations次
'''
train_data,test_data = load_data()
x = train_data[:,:-1]
y = train_data[:,-1]

net = LinearNetwork(13)
num_iterations = 2000
losses = net.train(x,y,iterations=num_iterations,lr=0.01)

plot_x = np.arange(num_iterations)
plot_y = np.array(losses)
plt.plot(plot_x,plot_y)
plt.show()       
'''
# (8) SGD(使用mini-batch方法)
#在上述程序中，每次损失函数和梯度计算都是基于数据集中的全部数据，但是对于大容量样本的数据集，每次参数更新都使用全部数据会使得计算效率非常低。
#mini_batch:每次迭代抽取出的一批数据称为一个mini-batch
def load_data_SGD():

    batch_size = 10

    train_data,test_data = load_data()
    np.random.shuffle(train_data)

    mini_batches = [train_data[k:k+batch_size] for k in range(0,len(train_data),batch_size)]
    return mini_batches

class LinearNetwork_SGD(object):
    def __init__(self,num_of_weights):
        #随机初始化权重
        np.random.seed(2022)
        self.w = np.random.randn(num_of_weights,1)
        self.b = 0
    def forward(self,x):
        z = np.dot(x,self.w) + self.b
        return z
    def loss(self,z,y):#残差平方损失函数
        #使用平方误差而不是绝对值有两个好处
            #1,曲线的最低点是可导的
            #2,越接近最低点，坡度逐渐放缓，有助于通过当前的梯度值判断接近最低点的程度
        error = z.squeeze() - y.squeeze()
        cost = np.sum(error*error)/error.shape[0]

        return cost

    def gradient(self,x,y):
        z = self.forward(x)
        gradient_w_tmp = np.dot((z.squeeze()-y),x)
        #print(gradient_w_tmp.shape)
        gradient_w = gradient_w_tmp/x.shape[0]
        gradient_b = np.mean(z-y)
        return gradient_w,gradient_b
    def update(self,gradient_w,gradient_b,lr):
        self.w = self.w.squeeze() - lr*gradient_w.squeeze()
        self.b = self.b - lr*gradient_b

    def train(self,num_epochs,batch_size,lr):
        n = 404
        losses = []
        for epoch in range(num_epochs):
            mini_batches = load_data_SGD()
            for iter,mini_batch in enumerate(mini_batches):
                x = mini_batch[:,:-1]
                y = mini_batch[:,-1]
                z = self.forward(x)
                loss = self.loss(z,y)
                gradient_w,gradient_b = self.gradient(x,y)
                self.update(gradient_w,gradient_b,lr=0.01)
                losses.append(loss)
                print('iter:{} of epoch:{}: loss:{}'.format(epoch,iter,loss))
        return losses
    
net = LinearNetwork_SGD(13)
losses = net.train(50,10,0.01)#40*50=2000
plot_x = np.arange(len(losses))
plot_y = np.array(losses)
plt.plot(plot_x,plot_y)
plt.show()

iter:0 of epoch:0: loss:2.2345300046313477
iter:0 of epoch:1: loss:1.6970206220391038
iter:0 of epoch:2: loss:1.0467343732853
iter:0 of epoch:3: loss:1.6628371792032337
iter:0 of epoch:4: loss:1.441134310777762
iter:0 of epoch:5: loss:1.5239516787217393
iter:0 of epoch:6: loss:1.739225536395752
iter:0 of epoch:7: loss:1.7576329005945586
iter:0 of epoch:8: loss:1.6940953401760865
iter:0 of epoch:9: loss:1.817939488307556
iter:0 of epoch:10: loss:1.8290803017518784
iter:0 of epoch:11: loss:1.6583785573956162
iter:0 of epoch:12: loss:1.4389650424676246
iter:0 of epoch:13: loss:1.521185091630357
iter:0 of epoch:14: loss:1.3624862454097246
iter:0 of epoch:15: loss:1.3695605635644308
iter:0 of epoch:16: loss:1.2771501757166654
iter:0 of epoch:17: loss:1.0037817804303306
iter:0 of epoch:18: loss:1.5575464710794287
iter:0 of epoch:19: loss:1.095856996023564
iter:0 of epoch:20: loss:1.132534877668208
iter:0 of epoch:21: loss:1.2181318746069247
iter:0 of epoch:22: loss:2.162389608224804
iter:0 o

<Figure size 640x480 with 1 Axes>

### 总结:
### 在一个epoch下，迭代iter个mini_batch，算是遍历了一次数据集。
### iter = len(train_data) // batch_size

## 深度学习框架

Paddle框架在2016年正式开源


|思考过程|工作内容|个性化部分|通用部分(平台框架负责)|
|--|--|--|--|
|Step1: 模型设计|假设一种网络|设计网络结构|网络模块的实现(Layer,Variable)|
||设计评价函数|指定评价函数|Loss函数实现|
||寻找优化寻解方法|指定优化器|优化算法实现|
|Step2：准备数据集|准备训练数据|提供数据格式、位置、模型接受数据方式|为模型批量送入数据|
|Step3: 训练配置|训练配置|单机和多机配置|单机到多机的转换(transpile to run)|
|Step4: 应用部署|部署应用和测试环境|确定保存和加载模型的环节点|save_inference_model,load_inference_model|
|Step5: 模型评估|评估模型效果|指定评估指标|模型实现，Visual DL|
|Step6：基本过程|全流程串起来|主程序| |

- 飞桨集成 深度学习核心框架、基础模型库、端到端开发套件、工具组件、服务平台 于一体

![title](https://img2.baidu.com/it/u=68423699,3683953266&fm=253&fmt=auto&app=138&f=PNG?w=1071&h=500) 


| **框架工具** |简介|
|--|--|
|Paddle Inference|飞桨原生推理库,用于服务器端模型部署|
|Paddle Serving|飞桨服务化部署框架，用于云端服务器部署框架，可以将模型作为单独的Wed服务|
|Paddle Lite|飞桨轻量化推理模型，用于移动端部署。**部署到EdgeBoard就是用此方法**|
|Paddle.js|使用JS部署模型，在浏览器、小程序等环境部署模型|
|Paddle Slim|模型压缩工具|
|X2Paddle|飞桨模型转化工具，将其他框架模型转化为Paddle模型|
|AutoDL|飞桨自动搜索网络结构、超参数的工具|
|VisualDL|飞桨可视化分析工具|
|PaddleFL|飞桨联邦学习框架，让用户运用外部伙伴的服务器资源训练，但不泄露业务数据|
|PaddleX|飞桨全流程开发工具|

- Paddle Hub的使用最简单，模型库的可定制性最强，覆盖场景最简单
|Paddle Hub|预训练模型库、迁移学习组件。是进行FOC(模型验证的首选工具)| 
|--|--|

|**开发套件**|针对具体的应用场景提供的全套研发工具|
|--|--|
|PaddleClas|飞桨图像分类套件|
|PaddleDetection|飞桨目标检测套件|
|PaddleSeg|飞桨图像分割套件|
|PLSC|海量分类套件|
|Parakeet|飞桨语音合成套件|

> 移动端
>> ARM-CPU、Mali-CPU、Adreno-GPU、Metal GPU、FPGA、华为NPU

> 系统
>> Linux、MAC、Windows、Android、ios





## 2,使用飞桨重写波士顿房价预测
> step1 数据处理  
 step2 模型设计    
 step3 训练配置  
 step4 训练过程  
 step5 模型保存

 

In [3]:
import paddle
import paddle.fluid as fluid
import paddle.fluid.dygraph as dygraph
from paddle.fluid.dygraph import Linear
import numpy as np
import os
import random

# 1,数据处理
def load_data_SGD():

    batch_size = 10

    train_data,test_data = load_data()
    np.random.shuffle(train_data)

    mini_batches = [train_data[k:k+batch_size] for k in range(0,len(train_data),batch_size)]
    return mini_batches

# 2,模型设计
class Regressor(fluid.dygraph.Layer):
    def __init__(self):
        super(Regressor,self).__init__()
        self.fc = Linear(input_dim=13,output_dim=1,act=None)
    def forward(self,x):
        x = self.fc(x)
        return x
# 3,训练配置
# 训练配置包括四步: 指定运行的机器资源 ——> 声明模型实例 ——> 加载训练和测试数据 ——> 设置优化算法和学习率
with fluid.dygraph.guard():
#当with后面的代码块全部被执行完之后，将调用前面返回对象的 __exit__()方法
    model = Regressor()
    model.train() #开启训练模式
    train_data,test_data = load_data()
    opt = fluid.optimizer.SGD(learning_rate=0.01,parameter_list=model.parameters())

- 模型实例有两种形态:model.train()和model.eval()即训练和测试，训练正向反向都进行，测试只进行正向传播。且部分算子，如Drop out，BN 在这两种模式下执行方式不同。
- 模型实例化，读取数据，定义优化器 全部都在动态图上下文环境中进行

```python
'''
numpy底部使用C语言编写，运行非常快
'''
x_tenosr = dygraph.to_variable(x_numpy) #np转tensor
x_numpy = x_tensor.numpy() #tensor转numpy
```
##### ndarray切片
```python
a = np.arange(30)
b = a[4:7]
b[0] = 0
#此时a[4]也变成了0
'''
数组切片产生的新数组还是会指向原数组的内存区域，视图上任何的修改也会改变原数组
'''
a = np.arange(30)
b = np.copy(a[4:7])
b[0] = 0
```
##### np.random()
```python
np.random.seed(2022)
#生成符合均匀分布的数据
a = np.random.uniform(low=-1.0,high=1.0,size=(2,2))
#生成符合标准正态分布的数据
a = np.random.randn(3,3)
#随机打乱
np.random.shuffle(a)]
#随机选取元素
b = np.random.choice(a,size=5)
```

- 空格和回车都属于空白字符，即sep = ' '

In [4]:
# 4,训练过程
with dygraph.guard(fluid.CPUPlace()):
    EPOCH_NUM = 2000
    BATCH_SIZE = 10
    for epoch in range(EPOCH_NUM):
        mini_batches = load_data_SGD()

        for iter,mini_batch in enumerate(mini_batches):
            x = np.array(mini_batch[:,:-1]).astype('float32')
            y = np.array(mini_batch[:,-1]).astype('float32')

            x_p_tensor = dygraph.to_variable(x)
            y_p_tensor = dygraph.to_variable(y)

            predicts = model.forward(x_p_tensor)
            loss = fluid.layers.square_error_cost(predicts,label=y_p_tensor)
            avg_loss = fluid.layers.mean(loss)
            if iter % 10 == 0:
                print('iter:{} of epoch:{}: loss:{}'.format(iter,epoch,avg_loss))
            avg_loss.backward()
            opt.minimize(avg_loss)
            model.clear_gradients()
    # 保存模型
    fluid.save_dygraph(model.state_dict(),'Regressor')

iter:0 of epoch:0: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [1.63380551])
iter:10 of epoch:0: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [0.39020351])
iter:20 of epoch:0: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [0.19055939])
iter:30 of epoch:0: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [0.14655244])
iter:40 of epoch:0: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [0.43570739])
iter:0 of epoch:1: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [0.14061514])
iter:10 of epoch:1: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [0.14477594])
iter:20 of epoch:1: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [0.12468851])
iter:30 of epoch:1: loss:Tensor(shape=[1], dtype=float32, place=CPUPlace, 

KeyboardInterrupt: 

In [ ]:
# 5,模型测试
# 模型测试包括四步: 指定运行的机器资源 ——> 定义模型结构，加载模型参数 ——> 设置为eval模式 ——> 预测
_,test_data = load_data()
test_sample = test_data[10,:]#选择第十个样本进行测试
test_x = np.array(test_sample[:-1]).astype(np.float32)
test_y = test_sample[-1]

with dygraph.guard():
    model_dict,_ = fluid.load_dygraph('Regressor.pdparams')
    model.load_dict(model_dict)
    model.eval()
    test_x_p_tensor = dygraph.to_variable(test_x)
    results = model(test_x_p_tensor).numpy()
    print('label:{},label:{}   --(after BN)'.format(test_y,results[0]))


## 3,mnist(CIFAR)识别
- MNIST数据集是LeCun找250位不同标注员手写的
- paddle.dataset.cifar.train10(cycle=False)里的数据是(3,32,32)的
- 大量实现发现，模型对最后出现的数据印象更加深刻，训练数据导入后，越接近模型训练结束最后几个批次对模型的影响越大。为了避免模型记忆影响训练效果，每个epoch之前需要对数据进行乱序操作

#### 异步读取数据fluid.io.DataLoader.from_generator()
- 异步数据读取只是在数据规模巨大时会带来性能的显著提升，对于多数应用场景采用同步数据读取已经足够
- capacity表示都队列容量
- return_list在动态图模式下设置为True

```python
place = fluid.CPUPlace() #fluid.CUDAPlace()时，代表数据读取到GPU上
with fluid.dygraph.guard(place):
    trainset = paddle.dataset.cifar.train10(cycle=False) #enumerate(trainset())
    #train_reader = paddle.batch(trainset(),batch_size=8)
    data_loader = fluid.io.DataLoader.from_generator(capacity=5,return_list=True)
    '''
    创建一个Dataloader对象用于加载python生成器产生的数据，数据会由于python线程预先读取,异步送入到
    队列中
    '''
    data_loader.set_batch_generator(trainset,places=place)
    '''
    用创建的Dataloader对象设置一个数据生成器set_batch_generator,输入的参数是一个数据生成器和服务器
    象类型
    '''
```

- trainset是一个返回 all_img,all_label的函数，要用enumerate(trainset())
- data_loader.set_batch_generator(trainset,places=place)之后，要用enumerate(data_loader())

In [42]:
import paddle
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import Linear
import numpy as np
import os
from PIL import Image

# 1,数据预处理
# 飞桨支持很多常用的数据集，并把它们封装在paddle.dataset下(mnist,cifar,imdb...)
#对于容量大的数据集，我们使用异步读取的方式，即数据读取和模型训练并行
#把读取到的数据不断放入缓存，当训练完一个batch之后，自动读取下一个batch的数据
place = fluid.CPUPlace() #fluid.CUDAPlace()时，代表数据读取到GPU上

#检验下面的代码能不能正常运行，省的出错

with fluid.dygraph.guard(place):
    def load_data():
        def data_generator():
            trainset = paddle.dataset.cifar.train10(cycle=False) #enumerate(trainset())
            train_reader = paddle.batch(trainset,batch_size=100)
            for i,data in enumerate(train_reader()):
                #data是8*[3072,1]的数据
                img_batch = np.array([x[0].reshape((3,32,32)) for x in data]).astype('float32') #img_batch.shape=(8,3072)
                label_batch = np.array([np.array([x[1]]) for x in data]).astype('int64')
                yield np.array(img_batch),np.array(label_batch)
        return data_generator
'''
train_loader = load_data()
data_loader = fluid.io.DataLoader.from_generator(capacity=50,return_list=True)
data_loader.set_batch_generator(train_loader,places=place)
for batch_id,data in enumerate(data_loader):
    print(data[0].shape)
    img_batch,label_batch = data
    #img_batch.shape=(8,3,32,32)
    if batch_id > 2:
        break
'''

'\ntrain_loader = load_data()\ndata_loader = fluid.io.DataLoader.from_generator(capacity=50,return_list=True)\ndata_loader.set_batch_generator(train_loader,places=place)\nfor batch_id,data in enumerate(data_loader):\n    print(data[0].shape)\n    img_batch,label_batch = data\n    #img_batch.shape=(8,3,32,32)\n    if batch_id > 2:\n        break\n'

In [46]:
'''
展示图片 of cifar
for batch_id,data in enumerate(train_reader()):
    img_batch = np.array([x[0] for x in data]).astype('float32') #img_batch.shape=(8,3072)
    label_batch = np.array([x[1] for x in data]).astype('float32')
    break
k = 5
r,g,b = img_batch[k].reshape((3,32,32))[0][:,:,np.newaxis],img_batch[k].reshape((3,32,32))[1][:,:,np.newaxis],img_batch[k].reshape((3,32,32))[2][:,:,np.newaxis]
a = np.concatenate((r,g,b),axis=2)
import matplotlib.pyplot as plt
print(label_batch[k])
plt.imshow(a)
'''

"\n展示图片 of cifar\nfor batch_id,data in enumerate(train_reader()):\n    img_batch = np.array([x[0] for x in data]).astype('float32') #img_batch.shape=(8,3072)\n    label_batch = np.array([x[1] for x in data]).astype('float32')\n    break\nk = 5\nr,g,b = img_batch[k].reshape((3,32,32))[0][:,:,np.newaxis],img_batch[k].reshape((3,32,32))[1][:,:,np.newaxis],img_batch[k].reshape((3,32,32))[2][:,:,np.newaxis]\na = np.concatenate((r,g,b),axis=2)\nimport matplotlib.pyplot as plt\nprint(label_batch[k])\nplt.imshow(a)\n"

### 使用VisualDL
- ifro mvisualdl import LogWriter
- （1） log_writer = LogWriter('./log')
- （2） log_writer.add_scalar(tag='acc',step=iter,value=acc.numpy()) #iter是一个递增对象

- 如果没找到左侧工具栏中的可视化，在shell输入：
```python
visualdl --logdir ./random_log --port 8080
#之后就可以看到地址
```

In [52]:
from paddle.fluid.dygraph.nn import Conv2D
from paddle.fluid.dygraph.nn import Pool2D
from visualdl import LogWriter

# 2,模型设计
class SimpleCNN(fluid.dygraph.Layer):
    def __init__(self):
        super(SimpleCNN,self).__init__()
        self.conv1 = Conv2D(num_channels=3,num_filters=20,filter_size=5,stride=1,padding=2,act='relu')
        self.pool1 = Pool2D(pool_size=2,pool_stride=2,pool_type='max')
        self.conv2 = Conv2D(num_channels=20,num_filters=20,filter_size=5,stride=1,padding=2,act='relu')
        self.pool2 = Pool2D(pool_size=2,pool_stride=2,pool_type='max')
        '''使用以下代码查看Conv，Pool输出层的shape
        net = SimpleCNN()
        param_info = paddle.summary(net,input_size=(1,3,32,32),dtypes='float32')
        '''
        self.fc = Linear(input_dim=1280,output_dim=10,act='softmax')

    def forward(self,x,label):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = fluid.layers.reshape(x,[x.shape[0],-1])
        x = self.fc(x)
        acc = fluid.layers.accuracy(input=x,label=label)
        return x,acc

# 3，训练过程
with fluid.dygraph.guard(place):
    use_VisualDL = True#是否使用VisualDL
    if use_VisualDL:
        log_writer = LogWriter("./log")

    train_loader = load_data()
    model = SimpleCNN()
    model.train()
    #可以在optimizer中加入正则化
    #optimizer = fluid.optimizer.AdamOptimizer(learning_rate=0.00001,regularization=fluid.regularizer.L2Decay(regularization_coeff=0.1),parameter_list=model.parameters())
    optimizer = fluid.optimizer.SGDOptimizer(learning_rate=0.000001,parameter_list=model.parameters())

    EPOCH_NUM = 20
    log_iter = 0

    for epoch in range(EPOCH_NUM):
        data_loader = fluid.io.DataLoader.from_generator(capacity=50,return_list=True)
        data_loader.set_batch_generator(train_loader,places=place)
        for batch_id,data in enumerate(data_loader):
            img_batch,label_batch = data
            img_batch_tensor = fluid.dygraph.to_variable(img_batch)
            label_batch_tensor = fluid.dygraph.to_variable(label_batch)
            predict,acc = model(img_batch_tensor,label_batch_tensor)
            loss = fluid.layers.cross_entropy(predict,label_batch_tensor)
            avg_loss = fluid.layers.mean(loss)
            if (batch_id+1)% 100 == 0:
                if use_VisualDL:
                    log_writer.add_scalar(tag='acc',step=log_iter,value=acc.numpy())
                    log_writer.add_scalar(tag='loss',step=log_iter,value=avg_loss.numpy())
                    log_iter = log_iter + 100
                print('batch:{} of epoch:{},loss:{},acc:{}'.format(batch_id+1,epoch,avg_loss.numpy(),acc.numpy()))
            avg_loss.backward()
            optimizer.minimize(avg_loss)
            model.clear_gradients()
    fluid.save_dygraph(model.state_dict(),'simplecnn_of_cifar')



batch:100 of epoch:0,loss:[3.1957912],acc:[0.15]
batch:200 of epoch:0,loss:[3.525582],acc:[0.09]
batch:300 of epoch:0,loss:[3.4123232],acc:[0.13]
batch:400 of epoch:0,loss:[3.1204336],acc:[0.11]
batch:500 of epoch:0,loss:[3.1946807],acc:[0.12]
batch:100 of epoch:1,loss:[3.0950537],acc:[0.16]
batch:200 of epoch:1,loss:[3.4093294],acc:[0.09]
batch:300 of epoch:1,loss:[3.3155084],acc:[0.13]
batch:400 of epoch:1,loss:[3.025226],acc:[0.11]
batch:500 of epoch:1,loss:[3.1037273],acc:[0.12]
batch:100 of epoch:2,loss:[3.014048],acc:[0.16]
batch:200 of epoch:2,loss:[3.3139253],acc:[0.1]
batch:300 of epoch:2,loss:[3.2360172],acc:[0.12]
batch:400 of epoch:2,loss:[2.9480615],acc:[0.12]
batch:500 of epoch:2,loss:[3.0288596],acc:[0.12]
batch:100 of epoch:3,loss:[2.9467843],acc:[0.15]
batch:200 of epoch:3,loss:[3.2332547],acc:[0.09]
batch:300 of epoch:3,loss:[3.1687624],acc:[0.13]
batch:400 of epoch:3,loss:[2.8838072],acc:[0.12]
batch:500 of epoch:3,loss:[2.9655874],acc:[0.14]
batch:100 of epoch:4,los

KeyboardInterrupt: 

- 对于分类问题，直接以标签和概率作比较也不合理，绝大多数图像分类任务都使用交叉熵损失函数。
- fluid.layers.cross_entropy()要求输入是int64。

$$
L = -[\sum_{k = 1}^{n}t_{k}lgy_{k} + (1-t_{k})lg(1-y_{k})]
$$

- 由于损失函数本身的数学意义上的不同，我们无法对比损失值来对比那种损失函数效果更好，在图片分类问题中可以用的方法就只有分类准确率
- fluid.layers.accuracy(input=x,label=label)可以计算分类准确率,label是2D的

#### 资源配置:多卡训练、多机训练

```python
# 单卡训练
use_gpu = False
place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()

with fluid.dygraph.guard(place)

```

- 在工业实践中，训练一个在ImageNet上精度表现良好的模型，大概需要一周的时间。
> 我们采用分布式训练方法：(1)模型并行 (2)数据并行
>>(1)模型并行  
      将一个网络拆成多份，比如2012年的Alexnet就是采用模型并行的方式训练而成的。
      现常用在很大的，各模块相对独立的模型中，即模型过大无法完整的放入单个GPU。
> 
>>(2)数据并行
      每次读取多份数据，读取到的数据给多个设备上的相同模型，
      **飞桨用的就是数据并行方式的分布式训练**

#### 数据并行

- 1 我们需要一个梯度同步机制，梯度同步有两种方式：PRC通信，NCCL2通信
- 基本大家都是用的NCCL2(Nvidia Collective multi-GPU Communication Library)
![Parameter Server](https://ai-studio-static-online.cdn.bcebos.com/560af46fd88140e8bc357dfad0d21e547e4703073c834c6c99c342b79e5076e4)
![NCCL2](https://ai-studio-static-online.cdn.bcebos.com/a27f1873e4934a0f8cda436b33830268ef4621cf6b994deb839db0a272e75de1)
- PRC通信方式常用于**CPU**分布训练，他有两个节点：参数服务器(Parameter server)和训练节点(Trainer)
参数服务器收集来自各个设备的梯度更新信息，并计算出一个全局的梯度更新，Trainer用于训练，每个Trainer上的程序相同，但数据不同，当参数服务器收到来自训练节点的梯度更新请求时，统一的更新全局的梯度。
- NCCL2方式不需要启动Parameter Server进程，每个Trainer进程保存一份完整的模型参数，在完成梯度计算之后通过Trainer相互通信，Reduce之间的相互通信，Reduce梯度数据到所有结点的所有设备，然后每个节点再各自完成参数更新。

#### 在
> place  
 模型实例化  
 train_loader定义  
 loss聚合
 
#### 四个方面做出改变即可  
  
  


```python
# 1,从环境变量获取设备的ID
device_id = fluid.dygraph.parallel.Env().dev_id
place = fluid.CUDAPlace(device_id)

# 2,对定义的网络做预处理，设置为并行模式
strategy = fluid.dygraph.parallel.prepare_context()
model = SimpleCNN()
model = fluid.dygraph.parallel.DataParallel(model,strategy)

# 3,定义多GPU训练的reader
valid_loader = paddle.batch(paddle.dataset.mnist.test(),batch_size=16,drop_last=True)
valid_loader = fluid.contrib.reader.distributed_batch_reader(valid_loader)

# 4,收集每批次训练的loos,并聚合参数的梯度
avg_loss = mnist.scale_loss(avg_loss)
avg_loss.backward()
mnist.apply_collective_grads()
```

  
- 训练集：用于训练模型的参数
- 验证集：用于模型超参数的选择，比如网络结构的调整，正则化权重的选择。
- 测试集：用于模拟模型在应用后的真实效果.因为测试集没有参与任何模型优化或参数训练工作，所以它对模型来说是完全未知的样本，在不调整超参数或者优化网络结构时，测试集和验证集的效果是类似的，均能更真实的反应模型效果。


### 模型参数及优化器保存

- 1,使用动态学习率
```pytho1
lr = fluid.dygraph.PolynominalDecay(0.01,total_steps,0.00001)
```
- 2,保存模型的参数和优化器的参数

```python
fluid.save_dygraph(model.state_dict(),'./checkpoint/epoch{}'.format(epoch_id))
fluid.save_dygraph(optimizer.satte_dict(),'./checkpoint/epoch{}'.format(epoch_id))
# .
# .
# .
params_dict,opt_dict = fluid.load_dygraph(para,s_path)
                   
```